In [ ]:
# Workaround to install pystruct.
!pip install --upgrade pip setuptools==57.5.0 
!git clone https://github.com/pystruct/pystruct && cd pystruct
!curl https://patch-diff.githubusercontent.com/raw/pystruct/pystruct/pull/221.patch | git apply
!pip install pystruct/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.3 MB/s 
     |████████████████████████████████| 819 kB 48.0 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Cloning into 'pystruct'...
remote: Enumerating objects: 6974, done.
remote: Total 6974 (delta 0), reused 0 (delta 0), pack-reused 6974
Receiving objects: 100% (6974/6974), 12.16 MiB | 20.61 MiB/s, done.
Resolving deltas: 100% (4357/4357), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1260k    0 1260k    0     0  3036k      0 --:--:-- --:--:-- --:--:-- 3029k
<stdin>:912: trailing whitespace.
 * 
<stdin>:915: trailing whitespace.
 * 
<stdin>:2642: trailing whitespace.
      __pyx_t_3 = __Pyx_TypeCheck(__pyx_v_arg, __pyx_v_ndarray); 
<stdin>:2652: trailing whitespace.
      __pyx_t_2 = __pyx_memoryview_check(__pyx_v_arg); 
<stdin>:2660: trailing whitespace.
        __pyx_t_3 = __Pyx_TypeCheck(__pyx_v_arg_base, __pyx_v_ndarray); 
error: src/utils.c: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simpl

In [ ]:
import os
import xml.etree.ElementTree as et
import json
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as pp
from google.colab import drive
from tqdm.notebook import tqdm
from google.colab import files

In [ ]:
# Take Input
uploaded = files.upload()

Saving POD_0000.bmp to POD_0000.bmp


In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# A data structure that holds block of characters.
class PageBlock:
    def __init__(self, x1, y1, x2, y2, block_class='textRegion'):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.block_class = block_class

In [ ]:
# Part 1 - PreProcessing

# A utility function to remove margin
def shrink_to_best_fit(img, page_block):

    shrink_line = np.all(
        img[page_block.y1:page_block.y2+1, page_block.x1:page_block.x2+1] == 0,
        axis=1
    )

    shrink_col = np.all(
        img[page_block.y1:page_block.y2+1, page_block.x1:page_block.x2+1] == 0,
        axis=0
    )

    for val in shrink_line:
        if val:
            page_block.y1 += 1
        else:
            break

    for val in reversed(shrink_line):
        if val:
            page_block.y2 -= 1
        else:
            break

    for val in shrink_col:
        if val:
            page_block.x1 += 1
        else:
            break

    for val in reversed(shrink_col):
        if val:
            page_block.x2 -= 1
        else:
            break

    return page_block


def get_processed_img(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, pros_img = cv2.threshold(
        gray_img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    return pros_img

def plot_array(array):
    pp.plot(array)
    pp.show()

def draw_rectangle(img, page_block_list):

    def get_color(block_class):
        if (block.block_class == 'figureRegion'):
            return (255, 0, 0)
        elif (block.block_class == 'formulaRegion'):
            return (255, 255, 0)
        elif (block.block_class == 'tableRegion'):
            return (0, 255, 255)
        else:
            return (157, 0, 255)

    img_copy = img.copy()
    for block in page_block_list:
        color = get_color(block.block_class)
        cv2.rectangle(
            img_copy,
            (block.x1, block.y1, block.x2-block.x1+1, block.y2-block.y1+1),
            color,
            1
        )

    cv2_imshow(img_copy)

def get_dilated_img(img, shape, iterations):
    # dilate the image to improve vertical projection.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, shape)
    dilate = cv2.dilate(img, kernel, iterations)
    # cv2_imshow(dilate)
    return dilate

def get_column_separator(dilated_img):

    #page_block = shrink_to_best_fit(
    #   dilated_img,
    #    PageBlock(0, 0, dilated_img.shape[1]-1, dilated_img.shape[0]-1)
    #)
    page_block = PageBlock(0, 0, dilated_img.shape[1]-1, dilated_img.shape[0]-1)

    vertical_projection = np.sum(
        dilated_img[page_block.y1:page_block.y2 +
                    1, page_block.x1:page_block.x2+1],
        axis=0
    )

    #plot_array(vertical_projection)

    # find minima in projection
    try:
        offset = int(page_block.x2/3)

        col_sep_list = np.where(
            vertical_projection[page_block.x1+offset:page_block.x2-offset] ==
            np.amin(vertical_projection[page_block.x1+offset:page_block.x2-offset])
        )
        # print(offset)
        # print(page_block.x1+col_sep_list[0][0])
        return offset+col_sep_list[0][0]
    except ValueError:
        return -1

def line_extraction(img, page_block):
    partition_list = []

    block_horizontal_projection = np.sum(
        img[page_block.y1:page_block.y2+1, page_block.x1:page_block.x2+1],
        axis=1
    )

    # plot_array(block_horizontal_projection)

    blank_pixel_line_count = 0
    y1 = page_block.y1

    for val in block_horizontal_projection:
        if val == 0:
            blank_pixel_line_count += 1
        else:
            if blank_pixel_line_count >= 1:
                partition_list.append(y1-1)
            blank_pixel_line_count = 0

        y1 += 1

    lines = []
    cur_y = page_block.y1

    for y in partition_list:

        page_line = shrink_to_best_fit(
            img,
            page_block=PageBlock(page_block.x1, cur_y, page_block.x2, y)
        )

        lines.append(page_line)
        cur_y = y+1

    page_line = shrink_to_best_fit(
        img,
        page_block=PageBlock(page_block.x1, cur_y,
                             page_block.x2, page_block.y2)
    )

    lines.append(page_line)

    return lines

def do_blocks_overlap(left_page_block, right_page_block):
    return not(
        left_page_block.y2 < right_page_block.y1 or
        left_page_block.y1 > right_page_block.y2
    )


def get_average_space(pros_img, page_block):
    dilate = get_dilated_img(
        pros_img[page_block.y1:page_block.y2+1, page_block.x1:page_block.x2+1],
        shape=(3, 3),
        iterations=2
    )

    vertical_projection = np.sum(dilate, axis=0)

    count = 0
    times_count = 0
    sum_of_space_count = 0

    for val in vertical_projection:
        if val == 0:
            count += 1
        else:
            if count != 0:
                sum_of_space_count += count
                count = 0
                times_count += 1

    if count != 0:
        sum_of_space_count += count
        count = 0
        times_count += 1

    if (times_count != 0):
        return sum_of_space_count/times_count
    else:
        # When times count = 0, that signifies there is no space in this line.
        # Here we will have to consider that the threshold is equal to the width
        # of a single space. Unlike the code above here we are calculating the 
        # width of a letter.
        vertical_projection = np.sum(
            pros_img[page_block.y1:page_block.y2 +
                     1, page_block.x1:page_block.x2+1],
            axis=0
        )

        count_of_letters = 0
        pixels_in_letter = 0
        sum_of_pixels_in_letters = 0

        for val in vertical_projection:
            if val != 0:
                pixels_in_letter += 1
            else:
                if pixels_in_letter != 0:
                    count_of_letters += 1
                    sum_of_pixels_in_letters += pixels_in_letter
                    pixels_in_letter = 0

        if pixels_in_letter != 0:
            count_of_letters += 1
            sum_of_pixels_in_letters += pixels_in_letter

        if count_of_letters == 0:
            return 0
        else:
            return sum_of_pixels_in_letters/count_of_letters

def develop_lines(pros_img, left_col_lines, right_col_lines):
    ordered_lines = []
    temp_left_col = []
    temp_right_col = []

    i = 0
    j = 0
    while(i < len(left_col_lines) and j < len(right_col_lines)):
        left_page_block = left_col_lines[i]
        right_page_block = right_col_lines[j]

        if (do_blocks_overlap(left_page_block, right_page_block)):

            # Average space is considered to be equal to average of space of left
            # line + average of space of right line i.e. approx 2 times the width of 
            # actual space width.
            average_space = (get_average_space(pros_img, left_page_block) + get_average_space(pros_img, right_page_block))
            short_height_block_difference = min(abs(left_page_block.y2-left_page_block.y1),abs(right_page_block.y2-right_page_block.y1))
            long_height_difference = max(abs(left_page_block.y2-left_page_block.y1),abs(right_page_block.y2-right_page_block.y1))
            
            if (right_page_block.x1 - left_page_block.x2 <= average_space):
              if(short_height_block_difference >= (long_height_difference)/2):
                  ordered_lines += temp_left_col + temp_right_col

                  temp_left_col.clear()
                  temp_right_col.clear()
                  ordered_lines.append(
                        PageBlock(
                            x1=left_page_block.x1,
                            y1=min(left_page_block.y1, right_page_block.y1),
                            x2=right_page_block.x2,
                            y2=max(left_page_block.y2, right_page_block.y2)
                        )
                    )
                  i+=1
                  j+=1
              #Height Difference is more 
              else :
                  if(left_page_block.y2 < right_page_block.y2):
                    temp_left_col.append(left_page_block)
                    i += 1
                  else:
                    temp_right_col.append(right_page_block)
                    j += 1
            #Average Space is not matched
            else :
                if(left_page_block.y2 < right_page_block.y2):
                    temp_left_col.append(left_page_block)
                    i += 1
                else:
                    temp_right_col.append(right_page_block)
                    j += 1

        # No OverLapping  
        elif (left_page_block.y2 < right_page_block.y1):
            temp_left_col.append(left_page_block)
            i += 1
        else:
            temp_right_col.append(right_page_block)
            j += 1

    while (i < len(left_col_lines)):
        temp_left_col.append(left_col_lines[i])
        i += 1

    while (j < len(right_col_lines)):
        temp_right_col.append(right_col_lines[j])
        j += 1

    ordered_lines += temp_left_col + temp_right_col

    return ordered_lines


def do_preprocessing(filename):

    img = cv2.imread(filename)

    pros_img = get_processed_img(img)

    dilate = get_dilated_img(pros_img, shape=(5, 5), iterations=4)

    col_separator = get_column_separator(dilate)

    if (col_separator == -1):
        return []

    left_page_block = shrink_to_best_fit(
        pros_img,
        page_block=PageBlock(0, 0, col_separator, pros_img.shape[0])
    )

    right_page_block = shrink_to_best_fit(
        pros_img,
        page_block=PageBlock(col_separator+1, 0,
                             pros_img.shape[1], pros_img.shape[0])
    )

    left_col_lines = line_extraction(pros_img, left_page_block)
    right_col_lines = line_extraction(pros_img, right_page_block)

    human_read_lines = develop_lines(pros_img, left_col_lines, right_col_lines)

    #draw_rectangle(img, human_read_lines)

    return human_read_lines


In [ ]:
img = cv2.imread('POD_0000.bmp')
#cv2_imshow(image_name)

In [ ]:
from pystruct.utils import SaveLogger
saved_unary_model_path = "drive/MyDrive/btp-dataset/SavedModels/unary/"
saved_pairwise_model_path = "drive/MyDrive/btp-dataset/SavedModels/pairwise/"
saved_crf_model_path = "drive/MyDrive/btp-dataset/SavedModels/crf/"
prepared_unannotation_folder_path = "drive/MyDrive/btp-dataset/Example/"

In [ ]:
unannotated_lines = do_preprocessing('POD_0000.bmp')
#print(unannotated_lines)
#draw_rectangle(image_name, unannotated_lines)
data = {}
data['lines'] = []
for line in unannotated_lines:
  data['lines'].append(
      {
          'x1': int(line.x1),
          'y1': int(line.y1),
          'x2': int(line.x2),
          'y2': int(line.y2),
          'block_class': line.block_class
      }
  )
# print(type(data))
json_file = prepared_unannotation_folder_path +'Result.json'
# print(json_file)
with open(json_file, 'w') as outfile:
    json.dump(data, outfile)


In [ ]:
uploaded = files.upload()

Saving Result.json to Result.json


In [ ]:
from tqdm.notebook import tqdm
import json
import cv2
import numpy as np
import os
from pystruct.models import EdgeFeatureGraphCRF
from pystruct.learners import FrankWolfeSSVM
from pystruct.utils import SaveLogger
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
from keras.metrics import sparse_categorical_crossentropy, categorical_crossentropy
from google.colab import drive



def scale_img_for_unary(img, x1, x2, y1, y2):

    width = x2 - x1 + 1
    height = y2 - y1 + 1

    final_img = img[y1 : y2 + 1, x1 : x2 + 1]

    if width <= 640 and height <= 32:
        vertical_padding = 32 - height
        horizontal_padding = 640 - width
        final_img = cv2.copyMakeBorder(
            final_img,
            vertical_padding // 2,
            vertical_padding // 2 + vertical_padding % 2,
            horizontal_padding // 2,
            horizontal_padding // 2 + horizontal_padding % 2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    elif height <= 32:
        scale_down = width / 640
        adjusted_height = int(height / scale_down)
        final_img = cv2.resize(final_img, (640, adjusted_height))

        vertical_padding = 32 - adjusted_height

        final_img = cv2.copyMakeBorder(
            final_img,
            vertical_padding // 2,
            vertical_padding // 2 + vertical_padding % 2,
            0,
            0,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    elif width <= 640:
        scale_down = height / 32
        adjusted_width = int(width / scale_down)
        final_img = cv2.resize(final_img, (adjusted_width, 32))

        horizontal_padding = 640 - adjusted_width

        final_img = cv2.copyMakeBorder(
            final_img,
            0,
            0,
            horizontal_padding // 2,
            horizontal_padding // 2 + horizontal_padding % 2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    else:
        scale_down_by_width = width / 640
        scale_down_by_height = height / 32

        if scale_down_by_height < scale_down_by_width:
            adjusted_height = int(height / scale_down_by_width)
            final_img = cv2.resize(final_img, (640, adjusted_height))
            vertical_padding = 32 - adjusted_height

            final_img = cv2.copyMakeBorder(
                final_img,
                vertical_padding // 2,
                vertical_padding // 2 + vertical_padding % 2,
                0,
                0,
                cv2.BORDER_CONSTANT,
                None,
                (255, 255, 255),
            )

        else:
            adjusted_width = int(width / scale_down_by_height)
            final_img = cv2.resize(final_img, (adjusted_width, 32))
            horizontal_padding = 640 - adjusted_width

            final_img = cv2.copyMakeBorder(
                final_img,
                0,
                0,
                horizontal_padding // 2,
                horizontal_padding // 2 + horizontal_padding % 2,
                cv2.BORDER_CONSTANT,
                None,
                (255, 255, 255),
            )

    return final_img


def scale_img_for_pairwise(img, x1, x2, y1, y2):

    width = x2 - x1 + 1
    height = y2 - y1 + 1

    final_img = img[y1 : y2 + 1, x1 : x2 + 1]

    if width <= 640 and height <= 16:
        vertical_padding = 16 - height
        horizontal_padding = 640 - width
        final_img = cv2.copyMakeBorder(
            final_img,
            vertical_padding // 2,
            vertical_padding // 2 + vertical_padding % 2,
            horizontal_padding // 2,
            horizontal_padding // 2 + horizontal_padding % 2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    elif height <= 16:
        scale_down = width / 640
        adjusted_height = int(height / scale_down)
        final_img = cv2.resize(final_img, (640, adjusted_height))

        vertical_padding = 16 - adjusted_height

        final_img = cv2.copyMakeBorder(
            final_img,
            vertical_padding // 2,
            vertical_padding // 2 + vertical_padding % 2,
            0,
            0,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    elif width <= 640:
        scale_down = height / 16
        adjusted_width = int(width / scale_down)
        final_img = cv2.resize(final_img, (adjusted_width, 16))

        horizontal_padding = 640 - adjusted_width

        final_img = cv2.copyMakeBorder(
            final_img,
            0,
            0,
            horizontal_padding // 2,
            horizontal_padding // 2 + horizontal_padding % 2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    else:
        scale_down_by_width = width / 640
        scale_down_by_height = height / 16

        if scale_down_by_height < scale_down_by_width:
            adjusted_height = int(height / scale_down_by_width)
            final_img = cv2.resize(final_img, (640, adjusted_height))
            vertical_padding = 16 - adjusted_height

            final_img = cv2.copyMakeBorder(
                final_img,
                vertical_padding // 2,
                vertical_padding // 2 + vertical_padding % 2,
                0,
                0,
                cv2.BORDER_CONSTANT,
                None,
                (255, 255, 255),
            )

        else:
            adjusted_width = int(width / scale_down_by_height)
            final_img = cv2.resize(final_img, (adjusted_width, 16))
            horizontal_padding = 640 - adjusted_width

            final_img = cv2.copyMakeBorder(
                final_img,
                0,
                0,
                horizontal_padding // 2,
                horizontal_padding // 2 + horizontal_padding % 2,
                cv2.BORDER_CONSTANT,
                None,
                (255, 255, 255),
            )

    return final_img


def get_unary_class(u):
    if u == "figureRegion":
        return 0
    elif u == "tableRegion":
        return 1
    elif u == "formulaRegion":
        return 2
    else:
        return 3


def get_unary_data_for_page(annotation_filename, cnn=False):
    train_samples = []
    train_labels = []

    #image_filename = annotation_filename[:-4] + "bmp"
    #img = cv2.imread(image_name)

    with open('Result.json', "r") as f:
        data = json.load(f)

        for line in data["lines"]:
            try:
                x1, y1, x2, y2 = line["x1"], line["y1"], line["x2"], line["y2"]
                scaled_line_img = scale_img_for_unary(img, x1, x2, y1, y2)
                train_samples.append(scaled_line_img)
                train_labels.append(get_unary_class(line["block_class"]))

                if cnn and get_unary_class(line["block_class"]) != get_unary_class(
                    "textRegion"
                ):
                    train_samples.append(cv2.flip(scaled_line_img, 0))
                    train_labels.append(get_unary_class(line["block_class"]))
                    train_samples.append(cv2.flip(scaled_line_img, 1))
                    train_labels.append(get_unary_class(line["block_class"]))
                    train_samples.append(cv2.flip(scaled_line_img, -1))
                    train_labels.append(get_unary_class(line["block_class"]))
            except Exception as e:
                print(e)
                pass

    return train_samples, train_labels


def get_pairwise_class(u, v):
    if v == "textRegion" and u == "tableRegion":
        return 0
    elif v == "textRegion" and u == "formulaRegion":
        return 1
    elif v == "textRegion" and u == "textRegion":
        return 2
    elif v == "textRegion" and u == "figureRegion":
        return 3
    elif v == "tableRegion" and u == "tableRegion":
        return 4
    elif v == "tableRegion" and u == "formulaRegion":
        return 5
    elif v == "tableRegion" and u == "textRegion":
        return 6
    elif v == "tableRegion" and u == "figureRegion":
        return 7
    elif v == "figureRegion" and u == "tableRegion":
        return 8
    elif v == "figureRegion" and u == "formulaRegion":
        return 9
    elif v == "figureRegion" and u == "textRegion":
        return 10
    elif v == "figureRegion" and u == "figureRegion":
        return 11
    elif v == "formulaRegion" and u == "tableRegion":
        return 12
    elif v == "formulaRegion" and u == "formulaRegion":
        return 13
    elif v == "formulaRegion" and u == "textRegion":
        return 14
    elif v == "formulaRegion" and u == "figureRegion":
        return 15
    else:
        return -1


def get_pairwise_data_for_page(annotation_filename):
    train_samples = []
    train_labels = []
    #image_filename = annotation_filename[:-4] + "bmp"
    #img = cv2.imread(image_name)

    with open('Result.json', "r") as f:
        data = json.load(f)
        line_list = data["lines"]

        for i in range(len(line_list) - 1):
            try:
                y11, x11 = line_list[i]["y1"], line_list[i]["x1"]
                y12, x12 = line_list[i]["y2"], line_list[i]["x2"]
                y21, x21 = line_list[i + 1]["y1"], line_list[i + 1]["x1"]
                y22, x22 = line_list[i + 1]["y2"], line_list[i + 1]["x2"]
                scaled_img1 = scale_img_for_pairwise(img, x11, x12, y11, y12)
                scaled_img = scale_img_for_pairwise(img, x21, x22, y21, y22)
                scaled_img1 = cv2.resize(img[y11 : y12 + 1, x11 : x12 + 1], (640, 16))
                scaled_img2 = cv2.resize(img[y21 : y22 + 1, x21 : x22 + 1], (640, 16))
                scaled_line_img = cv2.vconcat([scaled_img1, scaled_img2])

                train_samples.append(scaled_line_img)
                train_labels.append(
                    get_pairwise_class(
                        line_list[i]["block_class"], line_list[i + 1]["block_class"]
                    )
                )
            except Exception as e:
                pass
    return train_samples, train_labels


def get_unary_potentials(x):
    unary_potentials = []
    unary = tf.keras.models.load_model(saved_unary_model_path)
    for t in x:
        t = np.expand_dims(t, axis=0)
        p = unary.predict(t, verbose=0)
        unary_potentials.append(np.array(p[0]))
        # print(np.shape(p))
    return unary_potentials


def get_pairwise_potentials(x):
    pairwise_potentials = []
    pairwise = tf.keras.models.load_model(saved_pairwise_model_path)
    for t in x:
        t = np.expand_dims(t, axis=0)
        pairwise_potentials.append(np.array(pairwise.predict(t, verbose=0)[0]))
    return pairwise_potentials


# def get_crf_training_data():
#     # We have 1600 files, first 1280 are for training.
#     x_train = []
#     y_train = []

#     list_of_files = os.listdir(train_prepared_annotation_folder_path)
#     total_number_of_files = 300
#     file_counter = 0
#     with tqdm(total=total_number_of_files) as pbar:
#         for i in range(total_number_of_files):
#             annotation_filename = list_of_files[i]
#             file_counter += 1

#             x_train_u, y_train_u = get_unary_data_for_page(
#                 annotation_filename, cnn=False
#             )
#             x_train_p, _ = get_pairwise_data_for_page(annotation_filename)
#             unary_potential_list = np.array(get_unary_potentials(x_train_u))

#             pairwise_potential_list = np.array(get_pairwise_potentials(x_train_p))


#             n_nodes = len(unary_potential_list)
#             edges = np.transpose(
#                 np.vstack([np.arange(n_nodes - 1), np.arange(1, n_nodes)])
#             )

#             if (
#                 np.shape(unary_potential_list)[0] - 1
#                 == np.shape(pairwise_potential_list)[0]
#                 and np.shape(y_train_u)[0] == np.shape(unary_potential_list)[0]
#             ):

#                 x_train.append((unary_potential_list, edges, pairwise_potential_list))
#                 y_train.append(np.array(y_train_u))

#             pbar.update(1)

#     return x_train, y_train

In [ ]:
def get_crf_testing_data():
  # We have 1600 files, first 1280 are for training.
  x_test = []
  y_test = []

  list_of_files = os.listdir(prepared_unannotation_folder_path)
  total_number_of_files = len(list_of_files)
  file_counter = 0
  with tqdm(total=total_number_of_files) as pbar:
    for i in range(total_number_of_files):
      annotation_filename = list_of_files[i]
      file_counter += 1

      x_test_u, y_test_u = get_unary_data_for_page(annotation_filename, cnn=False)
      x_test_p, _ = get_pairwise_data_for_page(annotation_filename)
      unary_potential_list = np.array(get_unary_potentials(x_test_u))
      pairwise_potential_list = np.array(get_pairwise_potentials(x_test_p))
      
      n_nodes = len(unary_potential_list)
      edges = np.transpose(np.vstack([np.arange(n_nodes - 1), np.arange(1, n_nodes)]))
      if(np.shape(unary_potential_list)[0] - 1 == np.shape(pairwise_potential_list)[0] and np.shape(y_test_u)[0]==np.shape(unary_potential_list)[0]):

        x_test.append((unary_potential_list, edges, pairwise_potential_list))
        y_test.append(np.array(y_test_u))
    
      pbar.update(1)

  return x_test, y_test

In [ ]:
sv = SaveLogger(saved_crf_model_path+'crf_model')
ssvm = sv.load()
x_test, y_test = get_crf_testing_data()
score = ssvm.score(x_test, y_test)
print(score)
y = ssvm.predict(x_test)

  0%|          | 0/1 [00:00<?, ?it/s]

1.0


In [ ]:
print(y)
print(y_test)

In [ ]:
print(type(y))
print(y[0])
# print(np.shape(y[0]))
# print(np.shape(x_test[0][0]))

<class 'list'>
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [ ]:
print(data)

{'lines': [{'x1': 199, 'y1': 132, 'x2': 907, 'y2': 146, 'block_class': 'textRegion'}, {'x1': 212, 'y1': 195, 'x2': 895, 'y2': 332, 'block_class': 'textRegion'}, {'x1': 211, 'y1': 337, 'x2': 894, 'y2': 475, 'block_class': 'textRegion'}, {'x1': 539, 'y1': 488, 'x2': 558, 'y2': 504, 'block_class': 'textRegion'}, {'x1': 466, 'y1': 518, 'x2': 637, 'y2': 527, 'block_class': 'textRegion'}, {'x1': 382, 'y1': 532, 'x2': 722, 'y2': 752, 'block_class': 'textRegion'}, {'x1': 501, 'y1': 757, 'x2': 574, 'y2': 764, 'block_class': 'textRegion'}, {'x1': 539, 'y1': 780, 'x2': 558, 'y2': 796, 'block_class': 'textRegion'}, {'x1': 198, 'y1': 825, 'x2': 906, 'y2': 844, 'block_class': 'textRegion'}, {'x1': 197, 'y1': 851, 'x2': 907, 'y2': 868, 'block_class': 'textRegion'}, {'x1': 197, 'y1': 876, 'x2': 302, 'y2': 893, 'block_class': 'textRegion'}, {'x1': 198, 'y1': 936, 'x2': 364, 'y2': 953, 'block_class': 'textRegion'}, {'x1': 374, 'y1': 935, 'x2': 407, 'y2': 954, 'block_class': 'textRegion'}, {'x1': 228, 'y

In [ ]:
for files in y:
  for i in files:
    print(type(i))

In [ ]:
def label(img,data):
  list_of_boxes = len(y)
  i = 0
  for file in y:
    for label in file:
      if label == 0:
        data['lines'][i]['block_class'] = "figureRegion"
        i+=1
      elif label == 1:
        data['lines'][i]['block_class'] = "tableRegion"
        i+=1
      elif label == 2:
        data['lines'][i]['block_class'] = "formulaRegion"
        i+=1
      elif label == 3 :
        data['lines'][i]['block_class'] = "textRegion"
        i+=1

  labeled = prepared_unannotation_folder_path +'FinalResult.json'
  # print(labeled)
  with open(labeled, 'w') as outfile:
      json.dump(data, outfile)
      

In [ ]:
label(img,data)

In [ ]:
print(data)

{'lines': [{'x1': 199, 'y1': 132, 'x2': 907, 'y2': 146, 'block_class': 'textRegion'}, {'x1': 212, 'y1': 195, 'x2': 895, 'y2': 332, 'block_class': 'textRegion'}, {'x1': 211, 'y1': 337, 'x2': 894, 'y2': 475, 'block_class': 'textRegion'}, {'x1': 539, 'y1': 488, 'x2': 558, 'y2': 504, 'block_class': 'textRegion'}, {'x1': 466, 'y1': 518, 'x2': 637, 'y2': 527, 'block_class': 'textRegion'}, {'x1': 382, 'y1': 532, 'x2': 722, 'y2': 752, 'block_class': 'textRegion'}, {'x1': 501, 'y1': 757, 'x2': 574, 'y2': 764, 'block_class': 'textRegion'}, {'x1': 539, 'y1': 780, 'x2': 558, 'y2': 796, 'block_class': 'textRegion'}, {'x1': 198, 'y1': 825, 'x2': 906, 'y2': 844, 'block_class': 'textRegion'}, {'x1': 197, 'y1': 851, 'x2': 907, 'y2': 868, 'block_class': 'textRegion'}, {'x1': 197, 'y1': 876, 'x2': 302, 'y2': 893, 'block_class': 'textRegion'}, {'x1': 198, 'y1': 936, 'x2': 364, 'y2': 953, 'block_class': 'textRegion'}, {'x1': 374, 'y1': 935, 'x2': 407, 'y2': 954, 'block_class': 'textRegion'}, {'x1': 228, 'y

In [ ]:
pairwise = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape= (32,640,3)),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(16, activation='softmax')
])

pairwise.summary()

In [ ]:
pairwise.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])